In [29]:
import colorsys
from IPython.display import display, HTML

# --- 1. 헬퍼 함수들 (이전과 동일) ---

def hex_to_rgb_normalized(hex_code):
    hex_code = hex_code.lstrip('#')
    r, g, b = int(hex_code[0:2], 16), int(hex_code[2:4], 16), int(hex_code[4:6], 16)
    return (r / 255.0, g / 255.0, b / 255.0)

def rgb_normalized_to_hex(rgb_tuple):
    r, g, b = [int(val * 255) for val in rgb_tuple]
    r = max(0, min(255, r))
    g = max(0, min(255, g))
    b = max(0, min(255, b))
    return f"#{r:02x}{g:02x}{b:02x}"

def get_perceived_brightness(hex_code):
    r_norm, g_norm, b_norm = hex_to_rgb_normalized(hex_code)
    # Luma (Y') 공식
    brightness = (0.299 * r_norm) + (0.587 * g_norm) + (0.114 * b_norm)
    return brightness

# --- 2. (핵심) 목표 Luma 값을 찾는 함수 ---

def find_color_for_luma(target_h, target_s, target_luma, search_steps=500):
    """
    고정된 Hue(h), Saturation(s)을 가지면서 
    'target_luma' 값에 가장 가까운 HLS의 Lightness(l)를
    탐색하여 해당 색상의 Hex 코드를 반환합니다.
    """
    best_l = 0.0
    min_diff = 1.0 # Luma 값의 최대 차이는 1.0

    # 0.0부터 1.0까지 HLS의 'l' 값을 세밀하게 탐색
    for i in range(search_steps + 1):
        current_l = i / search_steps
        
        # HLS -> RGB 변환
        r, g, b = colorsys.hls_to_rgb(target_h, current_l, target_s)
        
        # RGB -> Luma(Y') 계산
        # (r,g,b가 0~1 범위이므로 정규화된 Luma 공식 바로 사용)
        current_luma = (0.299 * r) + (0.587 * g) + (0.114 * b)
        
        # 목표 Luma와의 차이 계산
        diff = abs(current_luma - target_luma)
        
        # 만약 이 l값이 목표 Luma에 더 가깝다면, 'best_l'로 저장
        if diff < min_diff:
            min_diff = diff
            best_l = current_l
            
    # 탐색이 끝난 후, 찾은 best_l을 사용해 최종 색상 생성
    final_r, final_g, final_b = colorsys.hls_to_rgb(target_h, best_l, target_s)
    return rgb_normalized_to_hex((final_r, final_g, final_b))

# --- 3. (새 함수) Luma 매칭 팔레트 생성기 ---

def generate_luma_matched_palettes(base_color1, base_color2, steps=10, min_y=0.35, max_y=0.95):
    """
    두 개의 기준 색상을 받아, 
    각 단계별 Luma(Y') 값이 일치하는 팔레트 두 개를 생성합니다.
    """
    palette1 = []
    palette2 = []
    
    # 1. 두 색상의 H, S 값을 고정
    h1, l1, s1 = colorsys.rgb_to_hls(*hex_to_rgb_normalized(base_color1))
    h2, l2, s2 = colorsys.rgb_to_hls(*hex_to_rgb_normalized(base_color2))

    # 2. 10단계의 '목표 Luma(Y') 값' 리스트를 생성
    target_lumas = [(max_y - min_y) * (i / (steps - 1)) + min_y for i in range(steps)]
    
    # 3. 각 목표 Luma 값에 대해 두 색상의 '최적의 l'값을 탐색
    for target_y in target_lumas:
        # 색상1에서 target_y(Luma)를 만족하는 색 탐색
        color1 = find_color_for_luma(h1, s1, target_y)
        palette1.append(color1)
        
        # 색상2에서 target_y(Luma)를 만족하는 색 탐색
        color2 = find_color_for_luma(h2, s2, target_y)
        palette2.append(color2)
        
    return palette1, palette2

# --- 4. 시각화 함수 (Luma 값 표시 기능 포함) ---
# (수정됨: 1열(세로)로 정렬되도록 수정)

def display_two_palettes_with_luma(title1, colors1, title2, colors2):
    """
    두 개의 색상 팔레트와 각 색상의 '인지적 밝기(Luma)' 점수를
    위아래(1열)로 나열하여 시각화합니다.
    """
    
    def generate_palette_html(title, colors):
        html = f'<div style="text-align: center; margin: 10px;">'
        html += f'<h3 style="font-family: sans-serif; color: #333;">{title}</h3>'
        html += '<div style="display: flex; flex-wrap: wrap; justify-content: center; width: 400px;">'
        
        for i, color_hex in enumerate(colors):
            luma = get_perceived_brightness(color_hex)
            text_color = "#000000" if luma > 0.4 else "#FFFFFF"
            
            html += f'''
            <div style="width: 100px; height: 100px; margin: 5px; 
                        background-color: {color_hex}; color: {text_color};
                        border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                        display: flex; flex-direction: column; 
                        justify-content: center; align-items: center;
                        font-family: sans-serif; font-weight: bold; font-size: 0.9em;">
                <span>{color_hex}</span>
                <span style="font-size: 0.8em; opacity: 0.8;">(Luma: {luma:.3f})</span>
            </div>
            '''
        html += '</div></div>'
        return html

    palette1_html = generate_palette_html(title1, colors1)
    palette2_html = generate_palette_html(title2, colors2)
    
    # --- 이 부분이 수정되었습니다 ---
    # flex-direction: column (세로 정렬) 및 align-items: center (가운데 정렬) 추가
    final_html = f'<div style="display: flex; flex-direction: column; align-items: center;">'
    # -----------------------------

    final_html += palette1_html
    final_html += palette2_html
    final_html += '</div>'
    
    display(HTML(final_html))

# --- 5. 메인 실행 코드 ---

# 🎨 이미지에서 추출한 두 색상
input_color_1 = "#FDAB7D" # (녹색 계열)
input_color_2 = "#F9D27B" # (주황색 계열)

print("--- 🎨 Luma(인지적 밝기) 기준 1:1 매칭 팔레트 ---")
print("각 팔레트의 같은 위치에 있는 색상은 Luma 점수가 거의 동일합니다.\n")

# Luma 값이 매칭되는 팔레트 생성 (7단계, min_y=0.25)
palette_1, palette_2 = generate_luma_matched_palettes(
    input_color_1, input_color_2, 
    steps=10
    
)

# 두 팔레트를 나란히 시각화
display_two_palettes_with_luma(
    f"Green Palette (Base: {input_color_1})", palette_1,
    f"Orange Palette (Base: {input_color_2})", palette_2
)

--- 🎨 Luma(인지적 밝기) 기준 1:1 매칭 팔레트 ---
각 팔레트의 같은 위치에 있는 색상은 Luma 점수가 거의 동일합니다.



In [33]:
import math

# --- 1. 하드웨어 누적 곱셈 시뮬레이션 함수 ---

def iterative_pow(base, exponent):
    """ (base)^exponent 를 누적 곱셈으로 계산 (Mul 1-6 방식 시뮬레이션) """
    if not math.isfinite(base): return float('nan')
    
    exponent = int(exponent) # 정수 지수
    if exponent == 0:
        return 1.0
    if exponent < 0: 
        return 1.0 / iterative_pow(base, -exponent) # 음수 지수 처리
    
    result = 1.0
    for _ in range(exponent):
        result *= base
    return result

def iterative_pow_cumulative(base, exponent):
    """ 
    (base)^(exp * (exp - 1) / 2) 를 누적 곱셈으로 계산 (Mul 7-15 방식 시뮬레이션) 
    P_k = (base)^(k(k-1)/2)
    재귀 관계: P(k) = P(k-1) * base^(k-1)
    """
    if not math.isfinite(base): return float('nan')
    exponent = int(exponent) # 정수 연산
    
    # P(0) = 1.0, P(1) = 1.0
    if exponent < 0: return float('nan') # 정의되지 않음
    if exponent == 0 or exponent == 1: return 1.0
    
    current_val = 1.0 # P(1)
    
    for k in range(2, exponent + 1):
        # P(k) = P(k-1) * base^(k-1)
        prev_k = k - 1
        
        # base^(k-1) 계산 (iterative_pow 사용)
        term_pow = iterative_pow(base, prev_k) 
        
        current_val = current_val * term_pow
        
    return current_val

def calculate_pu_approximation(x, E0_prime, Mx, M_dxx):
    """
    연산 B (하드웨어 근사 - 1D 전용): 
    PU(x) = E'_0 * (Mx)^x * (Mdxx)^(x(x-1)/2)
    (테이블 방식(누적 곱셈)으로 계산)
    """
    inputs = [E0_prime, Mx, M_dxx]
    if not all(math.isfinite(v) for v in inputs):
        return float('nan')

    try:
        # --- 누적 곱셈 수행 ---
        term_e0 = E0_prime
        
        # X항 (테스트 대상)
        term_x = iterative_pow(Mx, x)
        term_xx = iterative_pow_cumulative(M_dxx, x)
        
        return term_e0 * term_x * term_xx

    except (ValueError, OverflowError):
        return float('nan')

# --- 2. 물리 파라미터 -> 하드웨어 파라미터 변환 ---

def calculate_hw_parameters(center_x, center_y, sigma_xx, sigma_yy, sigma_xy, opacity):
    """
    물리적 가우시안 파라미터를 (0,0) 픽셀 기준 2차 테일러 근사를 통해 
    하드웨어 파라미터 (E0', M_x, ...)로 변환합니다.
    (1D X축 전용. log 중간 변수 사용 안 함)
    """
    
    # f(x, y) = log(Opacity) - 0.5 * (sigma_yy*(x-cx)^2 - 2*sigma_xy*(x-cx)*(y-cy) + sigma_xx*(y-cy)^2)
    
    cx = center_x
    cy = center_y
    
    try:
        # 1. f(0, 0) 계산 (E0'의 기반)
        f0_quad_term = (sigma_yy * (cx**2)) - (2.0 * sigma_xy * cx * cy) + (sigma_xx * (cy**2))
        f0 = math.log(opacity) - 0.5 * f0_quad_term
        
        # 2. 1차, 2차 도함수 계산 (Mx, Mdxx의 기반)
        df_dx_at_0 = (sigma_yy * cx) - (sigma_xy * cy)
        df_dxx_at_0 = -sigma_xx
        
        # 3. 최종 변환 (exp)
        hw_params = {
            "E0_prime": math.exp(f0),
            "Mx": math.exp(df_dx_at_0),
            "M_dxx": math.exp(df_dxx_at_0),
        }
        return hw_params
        
    except (ValueError, OverflowError):
        # e.g., math.log(0)
        return None

# --- 3. "원본" 2D 가우시안 계산 함수 (1D 슬라이스) ---

def calculate_ground_truth(x, center_x, center_y, sigma_xx, sigma_yy, sigma_xy, opacity):
    """
    연산 A (Ground Truth - 1D Slice at y=0):
    "원본" 2D 가우시안 공식을 (x, 0) 픽셀에 대해 직접 계산합니다.
    (근사식이 (0,0) 기준이므로 원본도 y=0 에서 비교)
    """
    
    # (0,0) 기준점이 아닌, (x, y) 픽셀 좌표 기준
    dx = float(x) - center_x
    dy = float(0) - center_y # y=0 고정
    
    try:
        # T = (sigma_yy * dx^2) - (2 * sigma_xy * dx * dy) + (sigma_xx * dy^2)
        term1 = sigma_yy * (dx ** 2)
        term2 = -2.0 * sigma_xy * dx * dy
        term3 = sigma_xx * (dy ** 2)
        
        T = (term1 + term2) + term3
        
        exponent = -0.5 * T
        return opacity * math.exp(exponent)
        
    except (ValueError, OverflowError):
        return float('nan')

# --- 4. 1D 리스트 출력 헬퍼 ---

def print_list(title, data_list):
    """ 1D 리스트를 보기 좋게 출력합니다. (x=0..7) """
    print(f"\n--- {title} (x=0..7) ---")
    print("x  |", end="")
    for x in range(len(data_list)):
        print(f"{x:^13}", end="")
    print("\n" + "-" * (4 + 13 * len(data_list)))
    print("val|", end="")
    for val in data_list:
        if math.isnan(val):
            print(f"{'NaN':>13}", end="")
        elif val == 0.0:
            print(f"{'0.0':>13}", end="")
        elif abs(val) < 1e-4 or abs(val) > 1e4:
            print(f"{val:13.4e}", end="")
        else:
            print(f"{val:13.6f}", end="")
    print()

# --- 5. 메인 실행 ---

if __name__ == "__main__":
    
    # --- *** 사용자가 수정할 입력 값 *** ---
    PHYSICAL_INPUTS = {
        "center_x": 1.5,      # 가우시안 중심 x' (픽셀 좌표 기준)
        "center_y": 2.2,      # 가우시안 중심 y' (픽셀 좌표 기준)
        "sigma_xx": 0.1,    # 공분산 행렬의 (1,1) (xx 아님, 수식 참고)
        "sigma_yy": 0.2,    # 공분산 행렬의 (0,0)
        "sigma_xy": 0.05,     # 공분산 행렬의 (0,1)
        "opacity": 0.9        # 가우시안 불투명도
    }
    # ------------------------------------
    
    # 1. 물리 파라미터를 하드웨어 파라미터로 1회 변환
    hw_params = calculate_hw_parameters(**PHYSICAL_INPUTS)
    
    if hw_params:
        # 2. 8-element 1D 리스트 계산 (근사 vs 원본)
        size = 8
        list_approx = [0.0] * size
        list_truth = [0.0] * size
        list_diff = [0.0] * size
        
        for x in range(size):
            # (A) X축 근사 하드웨어 로직
            val_approx = calculate_pu_approximation(x, **hw_params)
            
            # (B) "원본" 2D 가우시안 로직 (y=0 슬라이스)
            val_truth = calculate_ground_truth(x, **PHYSICAL_INPUTS)
            
            list_approx[x] = val_approx
            list_truth[x] = val_truth
            list_diff[x] = abs(val_approx - val_truth)
                
        # 3. 결과 리스트 출력
        print_list("계산된 1D 리스트 (X축 근사)", list_approx)
        print_list("원본 1D 리스트 (Ground Truth at y=0)", list_truth)
        print_list("오차 (근사 - 원본) (절대값)", list_diff)
        
    else:
        # 오류 발생 시 최소한의 알림
        print("[오류] 입력 값으로 인해 파라미터를 계산할 수 없습니다.")




--- 계산된 1D 리스트 (X축 근사) (x=0..7) ---
x  |      0            1            2            3            4            5            6            7      
------------------------------------------------------------------------------------------------------------
val|     0.665404     0.804640     0.796634     0.645739     0.428545     0.232850     0.103585     0.037728

--- 원본 1D 리스트 (Ground Truth at y=0) (x=0..7) ---
x  |      0            1            2            3            4            5            6            7      
------------------------------------------------------------------------------------------------------------
val|     0.665404     0.728068     0.652228     0.478375     0.287262     0.141231     0.056849     0.018735

--- 오차 (근사 - 원본) (절대값) (x=0..7) ---
x  |      0            1            2            3            4            5            6            7      
------------------------------------------------------------------------------------------------------------
val|